In [ ]:
import pandas as pd
import numpy as np
import chess
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import torch

Load in the data:

In [ ]:
file_path = 'chessData.csv'
data = pd.read_csv(file_path)
print(data.info())
data.head()

### Data Exploration

In [ ]:
print("Missing vals per column:\n", data.isnull().sum())


In [ ]:
# Remove outliers beyond a threshold
# this code: keep only evaluations within ±5000 centipawns. maybe double check where we should cut off outliers
# data = data[(data['Evaluation'] > -5000) & (data['Evaluation'] < 5000)]

In [ ]:
data['Evaluation'] = pd.to_numeric(data['Evaluation'], errors='coerce')

max_eval = data['Evaluation'].max()
min_eval = data['Evaluation'].min()

print(f"Maximum Evaluation: {max_eval}")
print(f"Minimum Evaluation: {min_eval}")

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data['Evaluation'], bins=50, kde=True)
plt.xlabel('Evaluation')
plt.ylabel('Frequency')
plt.title('Distribution of Evaluation Scores')
plt.show()

In [ ]:
non_numeric_values = data[pd.to_numeric(data['Evaluation'], errors='coerce').isna()]
print("Non-numeric values in Evaluation column:")
print(non_numeric_values[['Evaluation']])

### Data Cleaning

In [ ]:
# # means forced checkmate, could replace them with a really high positive score?:

# Replace '#+X' with a large positive value and '#-X' with a large negative value
data['Evaluation'] = data['Evaluation'].replace(
    {r'^\#\+.*': '10000', r'^\#\-.*': '-10000'}, regex=True
)


In [ ]:
# Drop NaN values
data.dropna(inplace=True)

non_numeric_values = data[pd.to_numeric(data['Evaluation'], errors='coerce').isna()]
print("Non-numeric values in Evaluation column:")
print(non_numeric_values[['Evaluation']])

In [ ]:
# train test split 

train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)


In [ ]:
import torch.nn as nn
import torch.optim as optim
# ! pip install torchvision
import torchvision
from torchvision import transforms
from itertools import islice
from torch.utils.data import DataLoader
from FEN_to_vector import to_vector
from torch.utils.data import Dataset

In [ ]:
class ChessDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): The DataFrame containing the FEN strings and target evaluations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get the FEN string and target evaluation
        fen = self.dataframe.iloc[idx]['FEN']
        evaluation = self.dataframe.iloc[idx]['Evaluation']

        # Use the 790-dimensional vector from FEN_to_vector
        board_vector = to_vector(fen)

        # Convert FEN to tensor (already in vector form)
        board_tensor = torch.tensor(board_vector, dtype=torch.float32)

        return board_tensor, torch.tensor(float(evaluation), dtype=torch.float32)


In [ ]:
portioned_train_data = train_data.sample(frac=0.3, random_state=42)
portioned_val_data = val_data.sample(frac=0.3, random_state=42)
portioned_test_data = test_data.sample(frac=0.3, random_state=42)

train_dataset = ChessDataset(portioned_train_data)
val_dataset = ChessDataset(portioned_val_data)
test_dataset = ChessDataset(portioned_test_data)

batch = 32
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=False)

In [ ]:
num_batches = len(train_loader)

print(f"Total number of batches: {num_batches}")

In [ ]:
class ChessNN(nn.Module):
    def __init__(self):
        super(ChessNN, self).__init__()
        # Define the fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(790, 512),  # Input layer (790 -> 512)
            nn.ReLU(),            # ReLU activation
            nn.Linear(512, 256),  # Hidden layer (512 -> 256)
            nn.ReLU(),            # ReLU activation
            nn.Linear(256, 1)     # Output layer (256 -> 1)
        )

    def forward(self, x):
        return self.fc_layers(x)

In [ ]:
# This is for me because I have a macbook :'(
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
from torch.optim.lr_scheduler import StepLR

model = ChessNN().to(device)
loss_fn = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001) # initial learning rate 0.001

lr_scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
batch_limit = 100000  # only process 10 batches 


model.eval()
val_predictions_before = []
val_targets_before = []

with torch.no_grad():
    for i, (board_tensors, targets) in enumerate(val_loader):
        if i >= batch_limit:
            break  # Stop after processing 'batch_limit' batches

        board_tensors, targets = board_tensors.to(device), targets.to(device)
        outputs = model(board_tensors)

        val_predictions_before.extend(outputs.squeeze().cpu().numpy())  # Store predictions
        val_targets_before.extend(targets.cpu().numpy())  # Store true targets

# Compute Mean Squared Error before training
mse_val_before = mean_squared_error(val_targets_before, val_predictions_before)
print(f"MSE before training: {mse_val_before:.4f}")


In [ ]:
epochs = 50  # Set to 1 for a quick test
batch_limit = 500 #100000  
losses = []
learning_rates = []

for epoch in range(epochs):
    model.train()  
    total_loss = 0
    current_lr = optimizer.param_groups[0]['lr']
    learning_rates.append(current_lr)

    for i, (board_tensors, targets) in enumerate(train_loader):
        if i >= batch_limit:
            break

        board_tensors, targets = board_tensors.to(device), targets.to(device)

        # Forward pass
        predictions = model(board_tensors)
        loss = loss_fn(predictions.squeeze(), targets)
        # losses.append(loss.item())

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    lr_scheduler.step()

    losses.append(total_loss / batch_limit)
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / batch_limit:.4f}")

**To run with entire data set for 10 epochs:**

In [ ]:
# To run with entire data set for 10 epochs:

# epochs = 10
# for epoch in range(epochs):
#     model.train()  # Set the model to training mode
#     total_loss = 0

#     for board_tensors, targets in train_loader:
#         # Move data to GPU if available
#         board_tensors, targets = board_tensors.to(device), targets.to(device)

#         # Ensure board_tensors has the right shape
#         if board_tensors.dim() == 3:
#             board_tensors = board_tensors.unsqueeze(0)

#         # Forward pass
#         predictions = model(board_tensors)
#         loss = loss_fn(predictions.squeeze(), targets)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}")

In [ ]:
# losses_per_epoch = [
#     np.mean(losses[i * batch_limit:(i + 1) * batch_limit])
#     for i in range(epochs)
# ]

fig, ax1 = plt.subplots(figsize=(10,6))

ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='tab:blue')
ax1.plot(range(epochs), losses, color='tab:blue', label='Loss')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Learning Rate', color='tab:red')
ax2.plot(range(epochs),learning_rates,color='tab:red',label='Learning Rate')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title('Loss over Training Epochs')
fig.tight_layout()
plt.show()

we want to consider how far off the score is rather than if it is exactly right

In [ ]:
model.eval()
val_predictions = []
val_targets = []

with torch.no_grad():
    for i, (board_tensors, targets) in enumerate(val_loader):
        if i >= batch_limit:
            break  # Stop after processing 'batch_limit' batches

        board_tensors, targets = board_tensors.to(device), targets.to(device)
        outputs = model(board_tensors)

        val_predictions.extend(outputs.squeeze().cpu().numpy())  # Store predictions
        val_targets.extend(targets.cpu().numpy())  # Store true targets

# Compute Mean Squared Error after evaluation
mse_val = mean_squared_error(val_targets, val_predictions)
print(f"Validation MSE: {mse_val:.4f}")


In [ ]:
model.eval()  # Set model to evaluation mode
val_loss = 0
batch_limit = 300  # Set the batch limit for validation

with torch.no_grad():  # No gradients are calculated during evaluation
    for i, (board_tensors, targets) in enumerate(val_loader):
        if i >= batch_limit:
            break

        board_tensors, targets = board_tensors.to(device), targets.to(device)

        # Forward pass
        predictions = model(board_tensors)
        loss = loss_fn(predictions.squeeze(), targets)
        val_loss += loss.item()  # Accumulate loss

print(f"Validation Loss: {val_loss / batch_limit:.4f}")
